Utilising a Python script to investigate deprivation locations and their relationships to football violence in England

First, we must import the external packages that we will be using to complete our script. The external packages that we are using are going to allow us to complete 
the desired functions for our analysis. 

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.features
import numpy as np
from matplotlib.patches import Patch

Next, we access the England deprivation areas shapefile. This will be utilised to provide an insight into the locations with varying levels of deprivation. To gauge 
an intial insight, we will define deprived_locations to sort the deprivation ranks of locations in decending order.  

In [ ]:
locations = gpd.read_file('data_files/England_Deprivation_Areas.shp')
deprived_locations = locations.sort_values(by='RAvgRank')

In [ ]:
print(deprived_locations.head(152))

After running this function -print(deprived_locations.head(152))-, we can see the first 5 and last 5 locations. An intial quick analysis allows us to realise that 4 of the most deprived locations are in the North of Englans, with 3 being located in the North West of England. Compared to the least deprived ranks, we can see that these values are within the South of England. 

Now, we will create the Stadiums data using a csv file which is located in the data_files folder. To do this, we will ustilse the pandas data frame to generate
our point data. We will also define the geometry using the longitude and latidue data in the csv file. After ensuring the set up of the geometry, we will set the coordinate system to EPSG:4326. 

In [ ]:
df = pd.read_csv('data_files/Stadiums_FBO.csv')
df['geometry'] = [Point(lon, lat) for lon, lat in zip(df['LONGITUDE'], df['LATITUDE'])]
stadiums = gpd.GeoDataFrame(df, geometry='geometry')
stadiums.crs = 'EPSG:4326'

After running this cell, we can then run the next cell to show the data that was created.

In [ ]:
stadiums.head

Here we can see the stadiums data of the 20 current Premier League teams with the specific data that we will be using throughout to complete analysis.